In [1]:
import os
os.getcwd()

'C:\\Users\\Nanda\\OneDrive - ANS\\Documentos\\ANS\\Bootcamp\\Projeto\\Telefonia'

In [42]:
import pandas as pd
resultados = pd.read_csv('./data/df_cm_ddd.csv', sep=';', encoding='utf-8')

In [3]:
resultados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6783 entries, 0 to 6782
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       6783 non-null   int64 
 1   Código Nacional  6783 non-null   int64 
 2   Grupo Econômico  6783 non-null   object
 3   media_ano        6783 non-null   object
 4   market_share     6783 non-null   object
 5   hhi_empresa      6783 non-null   object
 6   ano              6783 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 371.1+ KB


In [43]:
#Modificações Hélio

resultados['media_ano'] = resultados['media_ano'].map(lambda x: float(x.replace(',', '.')))

In [44]:
resultados = resultados[resultados['Grupo Econômico'] == 'TELECOM AMERICAS']

In [45]:
#criando dataframe com informações de georreferenciamento de municípios
georreferenciamento_df = pd.read_csv('https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv')

#os dados de georreferenciamento tem 7 dígitos (vamos remover o dígito verificador e atualizar o dataframe)
georreferenciamento_df['codigo_ibge'] = georreferenciamento_df['codigo_ibge']//10

In [8]:
georreferenciamento_df.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,520005,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,310010,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,520010,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,310020,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,150010,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [46]:
#cruzamento do dataframe resultados com as informações de georreferenciamento
resultados_df = pd.merge(resultados[['Código Nacional', 'ano', 'media_ano']],
                         georreferenciamento_df[['ddd', 'nome', 'codigo_ibge', 'codigo_uf', 'latitude', 'longitude']],
                         left_on='Código Nacional',
                         right_on='ddd',
                         how='inner')

In [10]:
resultados_df.head()

,Código Nacional,ano,media_ano,ddd,nome,codigo_ibge,codigo_uf,latitude,longitude
0,11,2009,5966137.66,11,Alumínio,350115,35,-23.5306,-47.2546
1,11,2009,5966137.66,11,Araçariguama,350275,35,-23.4366,-47.0608
2,11,2009,5966137.66,11,Arujá,350390,35,-23.3965,-46.3200
3,11,2009,5966137.66,11,Atibaia,350410,35,-23.1171,-46.5563
4,11,2009,5966137.66,11,Barueri,350570,35,-23.5057,-46.8790


In [47]:
#puxar a malha geográfica do brasil a nível de município
import requests
geojson = requests.get(f'http://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=municipio').json()

#a malha geográfica do ibge tem 7 dígitos (vamos remover o dígito verificador e atualizar a malha)
from geojson_rewind import rewind
for feature in geojson['features']:
    feature['properties']['codarea'] = feature['properties']['codarea'][:-1]
geojson = rewind(geojson, rfc7946=False)

In [48]:
max_min = resultados_df.groupby('ano').agg({'media_ano':'max'}).min().values[0]
max_min

6431194.75

In [49]:
resultados_df['media_ano'] = resultados_df['media_ano'].map(lambda x:max_min if x > max_min else x)

In [50]:
#construir o mapa choroplético com timeline no campo de ano
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

fig = px.choropleth(resultados_df,
                    geojson=geojson,
                    # scope='south america',
                    color='media_ano',
                    color_continuous_scale="Reds",
                    locations='codigo_ibge',
                    featureidkey='properties.codarea',
                    hover_name='nome',
                    animation_frame='ano')

fig.update_layout(height=800, width=1000, autosize=False)
fig.update_geos(fitbounds = "locations", visible = False)
fig.show();